In [ ]:
from graph import *
from ipywidgets import HTML
from ipyleaflet import Map, Polyline, Marker
import time

'''------------------------------------------------------------'''
'''-----------------------------Setup--------------------------'''
'''------------------------------------------------------------'''

print("Masukkan nama file:")
time.sleep(1)
file_name = input()
graph = Graph("../test/" + file_name)

vertices = graph.getVertices()
vertice_count = len(graph.getVertices())
adj_matrix = graph.getAdjMatrix()

center_lat = vertices[0].getCoordinate().getLat()
center_long = vertices[0].getCoordinate().getLong()
center = (center_lat, center_long)

'''------------------------------------------------------------'''
'''-------------------Bagian visualisasi graph-----------------'''
'''------------------------------------------------------------'''

map = Map(center = center, zoom = 16)

for vertice in vertices:
    marker_lat = vertice.getCoordinate().getLat()
    marker_long = vertice.getCoordinate().getLong()
    marker_name = vertice.getName()
    
    marker_message = HTML()
    marker_message.value = marker_name
    
    marker = Marker(location = (marker_lat, marker_long), draggable=False)
    marker.popup = marker_message
    
    map.add_layer(marker);
    
for i in range(vertice_count):
    for j in range(i, vertice_count):
        if adj_matrix[i][j] != 0:
            from_lat = vertices[i].getCoordinate().getLat()
            from_long = vertices[i].getCoordinate().getLong()
            dest_lat = vertices[j].getCoordinate().getLat()
            dest_long = vertices[j].getCoordinate().getLong()
            line = Polyline(
                locations=
                    [
                        [from_lat, from_long],
                        [dest_lat, dest_long]
                    ],
                color = "blue",
                fill = False
            )
            map.add_layer(line)
            
display(map)
time.sleep(1)

'''------------------------------------------------------------'''
'''-------------------Bagian visualisasi path------------------'''
'''------------------------------------------------------------'''

graph.showVerticesName()
validVertice = False
while (not validVertice):
    print("Masukkan nomor simpul asal:")
    from_vertice_idx = int(input())
    if (from_vertice_idx > 0 and from_vertice_idx <= vertice_count):
        validVertice = True
    else:
        print("Nomor simpul tidak valid")
validVertice = False
while (not validVertice):
    print("Masukkan nomor simpul tujuan:")
    dest_vertice_idx = int(input())
    if (dest_vertice_idx > 0 and dest_vertice_idx <= vertice_count and dest_vertice_idx != from_vertice_idx):
        validVertice = True
    else:
        print("Nomor simpul tidak valid")

path = graph.aStarPath(from_vertice_idx - 1, dest_vertice_idx - 1)

if path != None:
    map_path = Map(center = center, zoom = 16)
    
    path_matrix = graph.getPathMatrix(path)
    distance = graph.getPathDistance(path)
    
    for vertice in vertices:
        marker_lat = vertice.getCoordinate().getLat()
        marker_long = vertice.getCoordinate().getLong()
        marker_name = vertice.getName()

        marker_message = HTML()
        marker_message.value = marker_name

        marker = Marker(location = (marker_lat, marker_long), draggable=False)
        marker.popup = marker_message

        map_path.add_layer(marker);

    # Dua kali looping agar polyline merah ada di atas
    for i in range(vertice_count):
        for j in range(i, vertice_count):
            if adj_matrix[i][j] != 0 and path_matrix[i][j] != 1:
                from_lat = vertices[i].getCoordinate().getLat()
                from_long = vertices[i].getCoordinate().getLong()
                dest_lat = vertices[j].getCoordinate().getLat()
                dest_long = vertices[j].getCoordinate().getLong()
                line = Polyline(
                    locations=
                        [
                            [from_lat, from_long],
                            [dest_lat, dest_long]
                        ],
                    color = "blue",
                    fill = False
                )
                map_path.add_layer(line)
    
    for i in range(vertice_count):
        for j in range(i, vertice_count):
            if path_matrix[i][j] == 1:
                from_lat = vertices[i].getCoordinate().getLat()
                from_long = vertices[i].getCoordinate().getLong()
                dest_lat = vertices[j].getCoordinate().getLat()
                dest_long = vertices[j].getCoordinate().getLong()
                line = Polyline(
                    locations=
                        [
                            [from_lat, from_long],
                            [dest_lat, dest_long]
                        ],
                    color = "red",
                    fill = False
                )
                map_path.add_layer(line)

    display(map_path)
    time.sleep(1)
    
    print("Jalur yang ditemukan: " + path[0], end = "")
    for i in range(1, len(path)):
        print(" --> " + path[i], end = "")
    print("")
    print("Jarak: " + str(distance) + " meter")
else:
    print("Simpul tujuan tersebut tidak dapat dicapai dari simpul awal")